In [106]:
import sys

sys.path.append("../src")


import torch
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import sys
import os

In [108]:
import torch
from torchviz import make_dot

class CustomLinear(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.weight = torch.nn.Parameter(torch.randn(out_features, in_features))
        self.bias = torch.nn.Parameter(torch.randn(out_features))
        self.lr = torch.tensor(0.1, requires_grad=True)  # Обучаемый lr

        # История весов
        self.history_weight = self.weight.clone().detach()
        self.history_weight.requires_grad = True  # Чтобы градиенты проходили через него
        self.iteration = 0  # Счётчик итераций

    def forward(self, x):
        return torch.nn.functional.linear(x, self.history_weight, self.bias)

    def update_weights(self, loss):
        grad = torch.autograd.grad(loss, self.history_weight, create_graph=True)[0]  # Считаем градиент с сохранением графа
        self.history_weight = self.history_weight - self.lr * grad  # Обновляем историю

        # Каждые 10 итераций обновляем lr и сбрасываем историю
        self.iteration += 1
        if self.iteration % 10 == 0:
            grad_lr = torch.autograd.grad(loss, self.lr, retain_graph=True)[0]  # Градиент по lr
            self.lr = self.lr - 0.01 * grad_lr  # Обновление lr

            self.weight.data.copy_(self.history_weight.detach())  # Переносим историю в основное хранилище
            self.history_weight = self.weight.clone().detach()  # Сбрасываем историю
            self.history_weight.requires_grad = True  # Чтобы снова подключить её к графу

            # Визуализация графа после 10 итераций
            print(f"\nComputation graph for history_weight after {self.iteration} iterations:")
            make_dot(loss, params=dict(self.named_parameters())).render("computation_graph", format="png")  # Генерация изображения
            print("Graph saved as 'computation_graph.png'")

# Создаём слой и данные
layer = CustomLinear(10, 2)
x = torch.randn(10)
y = torch.randn(2)

# Тренировка
for i in range(30):  # Больше 10 итераций, чтобы проверить обновление lr
    output = layer(x)
    loss = ((output - y) ** 2).mean()

    layer.update_weights(loss)  # Обновляем веса через сохранённый граф

    print(f"Iter {i}, Loss: {loss.item()}, lr: {layer.lr.item()}")


Iter 0, Loss: 5.1314568519592285, lr: 0.10000000149011612
Iter 1, Loss: 0.15410013496875763, lr: 0.10000000149011612
Iter 2, Loss: 0.004627690650522709, lr: 0.10000000149011612
Iter 3, Loss: 0.00013897614553570747, lr: 0.10000000149011612
Iter 4, Loss: 4.172697117610369e-06, lr: 0.10000000149011612
Iter 5, Loss: 1.2524199632935051e-07, lr: 0.10000000149011612
Iter 6, Loss: 3.769801359965186e-09, lr: 0.10000000149011612
Iter 7, Loss: 1.1315848258419692e-10, lr: 0.10000000149011612
Iter 8, Loss: 3.690492356156483e-12, lr: 0.10000000149011612

Computation graph for history_weight after 10 iterations:
Graph saved as 'computation_graph.png'
Iter 9, Loss: 1.6264767310758543e-13, lr: 0.10000000149011612
Iter 10, Loss: 4.551914400963142e-15, lr: 0.10000000149011612
Iter 11, Loss: 4.551914400963142e-15, lr: 0.10000000149011612
Iter 12, Loss: 4.551914400963142e-15, lr: 0.10000000149011612
Iter 13, Loss: 4.551914400963142e-15, lr: 0.10000000149011612
Iter 14, Loss: 4.551914400963142e-15, lr: 0.10